# Usage Funnels with Warby Parker

<a href="https://www.warbyparker.com/">Warby Parker</a> is a transformative lifestyle brand with a lofty objective: to offer designer eyewear at a revolutionary price while leading the way for socially conscious businesses. Founded in 2010 and named after two characters in an early Jack Kerouac journal, Warby Parker believes in creative thinking, smart design, and doing good in the world. For every pair of eyeglasses and sunglasses sold, a pair is distributed to someone in need.

In this project, you will analyze different Warby Parker’s marketing funnels in order to calculate conversion rates. Here are the funnels and the tables that you are given:

Quiz Funnel:

- `survey`

Home Try-On Funnel:

- `quiz`
- `home_try_on`
- `purchase`

This project was a collaboration with Warby Parker’s Data Science team (thank you!) and uses fictional data.

## Quiz Funnel:

To help users find their perfect frame, Warby Parker has a Style Quiz that has the following questions:

1. “What are you looking for?”
2. “What’s your fit?”
3. “Which shapes do you like?”
4. “Which colors do you like?”
5. “When was your last eye exam?”

The users’ responses are stored in a table called survey.

In [ ]:
SELECT *
FROM survey
LIMIT 10;

Users will “give up” at different points in the survey. Let’s analyze how many users move from Question 1 to Question 2, etc.

In [ ]:
SELECt question, COUNT(DISTINCT user_id)
FROM survey
GROUP BY 1;

## Home Try-On Funnel:


Warby Parker’s purchase funnel is:

Take the Style Quiz → Home Try-On → Purchase the Perfect Pair of Glasses

During the Home Try-On stage, we will be conducting an A/B Test:

- 50% of the users will get 3 pairs to try on
- 50% of the users will get 5 pairs to try on
Let’s find out whether or not users who get more pairs to try on at home will be more likely to make a purchase.

The data will be distributed across three tables:

- quiz
- home_try_on
- purchase

Examine the first five rows of each table

In [ ]:
 SELECT *
 FROM quiz
 LIMIT 5;
 
SELECT *
FROM home_try_on
LIMIT 5;
 
 SELECT *
 FROM purchase
 LIMIT 5;

In [ ]:
SELECT DISTINCT q.user_id,
   h.user_id IS NOT NULL AS 'is_home_try_on',
   h.number_of_pairs,
   p.user_id IS NOT NULL AS 'is_purchase'
FROM quiz q
LEFT JOIN home_try_on h
   ON q.user_id = h.user_id
LEFT JOIN purchase p
   ON p.user_id = q.user_id
LIMIT 10;

In [ ]:
# We can calculate the difference in purchase rates between customers
# who had 3 number_of_pairs with ones who had 5.
WITH funnel as ( SELECT DISTINCT q.user_id, 
    h.user_id IS NOT NULL AS 'is_home_try_on',
    h.number_of_pairs,
    p.user_id IS NOT NULL AS 'is_purchase'
FROM quiz q
LEFT JOIN home_try_on h
    ON q.user_id = h.user_id
LEFT JOIN purchase p
    ON q.user_id = p.user_id)
SELECT number_of_pairs, SUM(is_purchase) AS 'num_purchase'
FROM funnel
GROUP BY 1
ORDER BY 1;

In [ ]:
# We can compare conversion from quiz→home_try_on and home_try_on→purchase.
WITH funnel as ( SELECT DISTINCT q.user_id, 
    h.user_id IS NOT NULL AS 'is_home_try_on',
    h.number_of_pairs,
    p.user_id IS NOT NULL AS 'is_purchase'
FROM quiz q
LEFT JOIN home_try_on h
    ON q.user_id = h.user_id
LEFT JOIN purchase p
    ON q.user_id = p.user_id)
SELECT COUNT(user_id), SUM(is_home_try_on), SUM(is_purchase),
    1.0 * SUM(is_home_try_on) / COUNT(user_id) AS 'quiz_to_home',
    1.0 * SUM(is_purchase) / SUM(is_home_try_on) AS 'home_to_purchase'
FROM funnel;

In [ ]:
# The most common results of the style quiz.
SELECT style, COUNT(DISTINCT user_id) AS 'count'
FROM quiz
GROUP BY 1
ORDER BY count DESC;